In [5]:
import gzip
from collections import defaultdict
import random
import string
import scipy.optimize
import scipy.stats
from sklearn import svm
import numpy as np
import pandas as pd

def readGz(f):
  for l in gzip.open(f):
    yield eval(l)

### Building the complete validation set   
user_list= []
business_list = []
ub_positive = []
for l in readGz("train.json.gz"):
  user,business = l['userID'],l['businessID']
  user_list.append(user)
  business_list.append(business)
  ub_positive.append((user,business,1))

ub_pos = ub_positive[:int(len(ub_positive)/2)]
ub_pair=list(zip(user_list,business_list))



d = defaultdict(list) 
for u,b in ub_pair:
  d[u].append(b)
    

ub_negative = []
ub_negative_dict=defaultdict(list)

count = 0
while(count < 100000):
  user_rand = user_list[random.randint(0,199999)]
  business_rand = business_list[random.randint(0,199999)]
  if business_rand in d[user_rand]:
    continue
  else:
    if business_rand not in ub_negative_dict[user_rand]:
      count += 1  
      ub_negative_dict[user_rand].append(business_rand)
      ub_negative.append((user_rand, business_rand,0))
    
ub_validation = ub_pos + ub_negative

print(len(ub_validation))

200000


In [39]:
import gzip
import collections
from collections import defaultdict
import random
import string
import scipy.optimize
import scipy.stats
from sklearn import svm
import numpy as np
import pandas as pd 

def readGz(f):
  for l in gzip.open(f):
    yield eval(l)

user_list_train= []
business_list_train = []
rating_list_train=[]
user_list_all= []
business_list_all = []
rating_list_all=[]
label_list=[]
data_train_pos=[]
data_train_neg=[]

businessCount = defaultdict(int)
userCount = defaultdict(int)
userCat = defaultdict(list)
businessCat = defaultdict(list)
businessUser = defaultdict(list)
I_u = {k:[] for k in range(794768)}
U_i = {k:[] for k in range(244412)}

itr=0
for l in readGz("train.json.gz"): 
  user_list_train.append(l['userID'])
  business_list_train.append(l['businessID'])
  rating_list_train.append(l['rating'])
  data_train_pos.append((l['userID'],l['businessID'],1))
  itr+=1
  if itr == 100000: break

        
for l in readGz("train.json.gz"): 
  u = l['userID']
  b = l['businessID']
  r = l['rating']
  cat = l['categories']
  user_list_all.append(u)
  business_list_all.append(b)
  rating_list_all.append(r)
  for c in cat:
    userCat[u].append(c)
    businessCat[b].append(c)
  userCount[u] += 1
  businessCount[b] +=1
  businessUser[u].append(b)
  if b not in U_i:
    U_i[b] = [l]
  U_i[b].append(l)
  if u not in I_u:
    I_u[u] = [l]
  I_u[u].append(l)


ub_pair=list(zip(user_list_all,business_list_all))
br_pair=list(zip(business_list_all, rating_list_all))

d_u = defaultdict(list)
d_b = defaultdict(list)
b_r = defaultdict(list)
for u,b in ub_pair:
  d_u[u].append(b)
  d_b[b].append(u)

for b,r in br_pair:
  b_r[b].append(r)
    
ub_negative_dict=defaultdict(list)

count = 0
while(count < 100000):
  user_rand = user_list_all[random.randint(0,199999)]
  business_rand = business_list_all[random.randint(0,199999)]
  if business_rand in d_u[user_rand]:
    continue
  else:
    if business_rand not in ub_negative_dict[user_rand]:
      count += 1  
      ub_negative_dict[user_rand].append(business_rand)
      data_train_neg.append((user_rand, business_rand,0))
    
data_train=data_train_pos+data_train_neg

print(len(data_train))




200000


In [40]:
most_common = [(businessCount[b], b) for b in businessCount]
most_common.sort()
most_common.reverse()
popular_business = [w[1] for w in most_common[:500]]

In [43]:
def jacard_business(b1,b2):

  set_1 = set([u['userID'] for u in userBusiness[b1]])
  set_2 = set([u['userID'] for u in userBusiness[b2]])
  n = float(len(set_1.intersection(set_2)))
  denominator = float(len(set_1) + len(set_2) - n)
  if denominator == 0.0: return 0.0
  return n /denominator  

def jacard_user(u1,u2):
  set_1 = set([i['businessID'] for i in I_u[u1]])
  set_2 = set([i['businessID'] for i in I_u[u2]])
  n = float(len(set_1.intersection(set_2)))
  denominator = float(len(set_1) + len(set_2) - n)
  if denominator == 0.0: return 0.0
  return n /denominator 

def jacard(user, business):
  set_1 = userBusiness[user]
  set_2 = businessUser[business]
  p = 0.0007
  most_similar_value = 0.0
  for b in set_1:
    j = jacard_business(b,business)
    if j > most_similar_value:
        most_similar_value = j
  if most_similar_value > p:
    return 1
  else:
    return 0
    

def business_jacard_last(user, business):
    if user not in I_u or business not in U_i:
      return 1.0
    i=I_u[user][-1]
    val = jacard_business(i['businessID'], business)
    return val

def user_jacard_last(user, business):
    if user not in I_u or business not in U_i:
      return 1.0
    u=U_i[business][-1]
    val = jacard_user(u['userID'], user)
    return val
        
def pearson_user(u1, u2):
  R_i1_avg = np.mean([i['rating'] for i in I_u[u1]])
  R_i2_avg = np.mean([i['rating'] for i in I_u[u2]])
  R_intersection = [{'u1': i['rating'], 'u2': v['rating']} for i in I_u[u1] for v in I_u[u2] if i['businessID'] == v['businessID']]
  numerator = (sum([(r['u1'] - R_i1_avg) * (r['u2'] - R_i2_avg) for r in R_intersection])) * 1.0
  denominator = (sum([(r['u1'] - R_i1_avg) ** 2 for r in R_intersection]) * sum([(r['u2'] - R_i2_avg) ** 2 for r in R_intersection])) ** 0.5
  return 0.0 if denominator == 0.0 else numerator / denominator 


def pearson_business(b1, b2):
  avg1 = np.mean([u['rating'] for u in userBusiness[b1]])
  avg2 = np.mean([u['rating'] for u in userBusiness[b2]])
  intersection = [{'b1': u1['rating'], 'b2': u2['rating']} for u1 in userBusiness[b1] 
                  for u2 in userBusiness[b2] if u1['userID'] == u2['userID']]
  numer = (sum([(i['b1'] - avg1) * (i['b2'] - avg2) for i in intersection])) * 1.0
  denomin = (sum([(i['b1'] - avg1) ** 2 for i in intersection])
             * sum([(i['b2'] -avg2) ** 2 for i in intersection])) ** 0.5
  return 0.0 if denomin== 0.0 else numer/ denomin

def pearson(user, business):
  value = 0.0
  if user not in businessUser or business not in userBusiness:
    return value
  for i in businessUser[user]:
    pearson_val = pearson_business(i['businessID'], business)
    if  pearson_val > 0.5:
        value = pearson_val
        return value  
  return value

def business_pearson_last(user, business):
    if user not in I_u or business not in U_i:
      return 0.0
    i=I_u[user][-1]
    pearson_val = pearson_business(i['businessID'], business)
    return pearson_val


def user_pearson_last(user, business):
    if user not in I_u or business not in U_i:
      return 0.0
    u=U_i[business][-1]
    pearson_val = pearson_user(u['userID'], user)
    return pearson_val

def most_active_user_pearson(user,business):
    if user not in I_u or business not in U_i:
      return 0.0
    most_active_count = 0
    most_active_user = ''
    for u in U_i[business]:
      count = userCount[u['userID']]
      if  count > most_active_count:
        most_active_count = count
        most_active_user = u['userID']
    
    pearson_val = pearson_user(most_active_user, user)
    return pearson_val

def most_active_business_pearson(user,business):
    if user not in I_u or business not in U_i:
      return 0.0
    most_active_count1 = 0
    most_active_business=''
    for i in I_u[user]:
      count =  businessCount[i['businessID']]
      if  count > most_active_count1:
        most_active_count1 = count
        most_active_business = i['businessID']
    
    pearson_val = pearson_business(most_active_business, business)
    return pearson_val



In [46]:
random.shuffle(data_train)


pos = dict(zip(popular_business, range(len(popular_business))))
word_set = set(popular_business) 

def feature(d):
  f1=businessCount[d[1]]
  f2=userCount[d[0]]
  f3=pearson(d[0],d[1])
  feat = [1, f1, f2, f3]
  return feat

x_train = [feature(d) for d in data_train]
y_train = [l[2] for l in data_train]
x_valid = [feature(d) for d in ub_validation]
y_valid = [d[2] for d in ub_validation]
clf=svm.LinearSVC(C=0.1)
clf.fit(x_train, y_train)
valid_predict = clf.predict(x_valid)
acc = [(x==y) for (x,y) in zip(valid_predict, y_valid)]
acc = sum(acc)*1.0/len(acc)
print(acc)

done
done


LinearSVC(C=0.1, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

KeyboardInterrupt: 

In [47]:
x_test=[]
for l in open("pairs_Visit.txt"):
    if l.startswith("userID"):
        continue
    u,i = l.strip().split('-')
    x_test.append((u,i))

x_test_fea= [feature(d) for d in x_test]
test_predict = clf.predict(x_test_fea)
index = 0
predictions = open("predictions_Visit.txt", 'w')

for l in open("pairs_Visit.txt"):
  if l.startswith("userID"):
    #header
    predictions.write(l)
    continue
  u,i = l.strip().split('-')
  predictions.write(u + '-' + i + ","+str(test_predict[index])+"\n")
  index+=1

predictions.close()

print("done")

done
